# Data Preprocessing 

Based on findings from `01_explore_data.ipynb`, implementing comprehensive preprocessing pipeline.

## Preprocessing Steps

During one pass over the data, we will do the following preprocessing steps.

1. **Convert pseudo-RGB to grayscale** (automatic with cv2.IMREAD_GRAYSCALE)
2. **Resize to 224×224 pixels** (standardize for CNN input)
3. **Apply CLAHE** (enhance contrast + normalize quality variations)

Then, we can focus on some further cleanup after the initial preprocessing is complete.

4. **Remove duplicates** (if found during processing)
5. **Create train/val/test splits** (this is already done for us in the Kaggle dataset)
6. **Address class imbalance** (augmentation for no_tumor class)

**Note:** Outlier removal skipped - CLAHE normalizes quality variations.

In [1]:
import logging

# Configure basic logging to a file
logging.basicConfig(
    filename='duplicates.log',  # Name of the log file
    level=logging.INFO,  # Minimum logging level to capture (INFO, DEBUG, WARNING, ERROR, CRITICAL)
    format='%(asctime)s - %(levelname)s - %(message)s',  # Format of log messages
    filemode='a'  # 'a' for append (default), 'w' for overwrite
)


## Preprocessing Steps 1-5

In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import os
import pandas as pd
import hashlib
from sklearn.model_selection import train_test_split
import json

In [3]:
# Preprocessing configuration
config = {
    'target_size': (224, 224),
    'apply_clahe': True,
    'clahe_clip_limit': 2.0,
    'clahe_tile_size': (8, 8),
    'random_seed': 42
}

# Paths
raw_dataset = Path("../data/brain_tumor_dataset")
processed_dataset = Path("../data/processed")

# create processed dataset directory if it doesn't exist
processed_dataset.mkdir(parents=True, exist_ok=True)

# Splits & Classes
splits = ['Training','Testing']
classes = ['glioma_tumor', 'meningioma_tumor', 'pituitary_tumor', 'no_tumor']

# create subdirectories for splits and classes
for split in splits:
    for cls in classes:
        dir_path = processed_dataset / split / cls
        dir_path.mkdir(parents=True, exist_ok=True)

print("Configuration:")
for key, value in config.items():
    print(f"  {key}: {value}")

Configuration:
  target_size: (224, 224)
  apply_clahe: True
  clahe_clip_limit: 2.0
  clahe_tile_size: (8, 8)
  random_seed: 42


In [4]:
image_hashes = {}

In [5]:
def get_image_hash(image_path):
    """Compute Perceptual Hash of an image"""

    img = cv2.imread(str(image_path), cv2.IMREAD_GRAYSCALE)

    if img is None:
        raise ValueError(f"Failed to read image: {img}")

    img_hash = cv2.img_hash.pHash(img).tobytes()

    return img_hash

In [6]:
def preprocess_image(img_path, target_size=(224,224), equalization=True):
    """
    Apply all preprocessing steps to a single image.
    
    Args:
        img_path: Path to input image
        target_size: Target image size (height, width)
        hist_normalization: Whether to apply histogram equalization
        
    Returns:
        Preprocessed grayscale image (numpy array)
    """

    # read image as grayscale
    img = cv2.imread(str(img_path), cv2.IMREAD_GRAYSCALE)

    if img is None:
            raise ValueError(f"Failed to read image: {img_path}")
    
    # resize so all images are the same size
    img = cv2.resize(img, target_size, interpolation=cv2.INTER_CUBIC)

    # apply clahe equalization
    if equalization:
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
        img = clahe.apply(img)
        
    return img

    

In [7]:
def preprocess_and_save_dataset(raw_dir, target_dir, config, visualize_n_duplicates=0):
    """
    Process entire raw dataset and save to new folder
    
    Args:
        raw_dir: Path to raw dataset
        target_dir: Path to save preprocessed dataset
        config: Dictionary with preprocessing parameters

    Returns:
        Dictionary with processing statistics
    """

    raw_path = Path(raw_dir)
    target_path = Path(target_dir)

    if not target_path.exists():
        os.makedirs(target_path, exist_ok=True)

    duplicate_example = 0

    # total images counts all images processed
    # class_counts is a dictionary that holds image counts per class
    stats = {'total_images': 0, 
             'class_counts': {}, 
             'duplicate_count': 0,
             'failed': []
    }    

    for split in splits:
        print(f"Processing {split} set...")
        
        for cls in classes:
            print(f"  Class: {cls}")
            class_path_raw = raw_path / split / cls
            class_path_target = target_path / split / cls

            # iterate through each class folder
            for filename in os.listdir(class_path_raw):

                # skip hidden files and non-image files
                if filename.startswith('.') or not filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
                    continue


                try:
                    img_path_raw = class_path_raw / filename
                    img_path_target = class_path_target / filename

                    # preprocess image
                    img = preprocess_image(img_path_raw,
                                        target_size=config['target_size'],
                                        equalization=config['apply_clahe']
                    )

                    # compute image hash for duplicate detection
                    img_hash = get_image_hash(img_path_raw)

                    # # check for duplicates
                    # if img_hash in image_hashes.keys():
                    #     stats['duplicate_count'] += 1
                    #     logging.info(f"Duplicate found: {img_path_raw} is a duplicate of {image_hashes[img_hash]}")

                    #     # visualize duplicates if requested
                    #     if visualize_n_duplicates > 0 and duplicate_example < visualize_n_duplicates:

                    #         duplicate_example += 1
                    #         img1 = img
                    #         img2 = cv2.imread(str(image_hashes[img_hash]), cv2.IMREAD_GRAYSCALE) # duplicate image

                    #         plt.figure(figsize=(8,4))
                    #         plt.suptitle(f"Duplicate Pair: class {cls} {duplicate_example}")

                    #         plt.subplot(1,2,1)
                    #         plt.title(f"Image 1: {filename}")
                    #         plt.imshow(img1, cmap='gray')
                    #         plt.axis('off')

                    #         plt.subplot(1,2,2)
                    #         plt.title(f"Image 2: {image_hashes[img_hash].name}")
                    #         plt.imshow(img2, cmap='gray')
                    #         plt.axis('off')

                    #         plt.show()
                    #     continue

                    # save to target folder
                    if not class_path_target.exists():
                        os.makedirs(class_path_target, exist_ok=True)

                    cv2.imwrite(str(img_path_target), img)
                    stats['total_images'] += 1
                    if cls not in stats['class_counts']:
                        stats['class_counts'][cls] = 0
                    stats['class_counts'][cls] += 1

                    # store image hash
                    image_hashes[img_hash] = img_path_target # key: image hash, value: target filepath
                except Exception as e:
                    print(f"Failed to process image {img_path_raw}: {e}")
                    stats['failed'].append((str(filename), str(e)))

    return stats

In [8]:
stats = preprocess_and_save_dataset(raw_dataset, processed_dataset, config, visualize_n_duplicates=3)
print("Preprocessing complete!")
print(f"Total images processed: {stats['total_images']}")
print(f"Total duplicates found and skipped: {stats['duplicate_count']}")
for cls, count in stats['class_counts'].items():
    print(f"  {cls}: {count} images")

Processing Training set...
  Class: glioma_tumor
  Class: meningioma_tumor
  Class: pituitary_tumor
  Class: no_tumor
Processing Testing set...
  Class: glioma_tumor
  Class: meningioma_tumor
  Class: pituitary_tumor
  Class: no_tumor
Preprocessing complete!
Total images processed: 3264
Total duplicates found and skipped: 0
  glioma_tumor: 926 images
  meningioma_tumor: 937 images
  pituitary_tumor: 901 images
  no_tumor: 500 images


In [ ]:
def visualize_preprocessing_effects(sample_image_path):
    """
    Visualize the effects of preprocessing on a sample image.
    Args:
        sample_image_path: Path to a sample raw image
    Returns:
        None
    """

    raw_img = cv2.imread(str(sample_image_path), cv2.IMREAD_GRAYSCALE)

    preprocessed_img = preprocess_image(sample_image_path,
                                    target_size=config['target_size'],
                                    equalization=config['apply_clahe']
    )

    plt.figure(figsize=(8,4))
    plt.suptitle(f"Preprocessing Effects on: {sample_image_path.name}")
    plt.subplot(1, 2, 1)
    plt.title("Raw Image")
    plt.imshow(raw_img, cmap='gray')
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.title("Preprocessed Image")
    plt.imshow(preprocessed_img, cmap='gray')
    plt.axis('off')

    plt.show()

In [ ]:
# show a few examples of preprocessed images before & after
def display_preprocessed_examples(raw_dir, processed_dir, classes, num_examples=3):
    raw_path = Path(raw_dir)
    processed_path = Path(processed_dir)

    for cls in classes:
        class_path_raw = raw_path / 'Training' / cls
        class_path_processed = processed_path / 'Training' / cls

        example_files = os.listdir(class_path_raw)[:num_examples]

        for filename in example_files:
            raw_img_path = class_path_raw / filename
            visualize_preprocessing_effects(raw_img_path)

display_preprocessed_examples(raw_dataset, processed_dataset, classes, num_examples=1)

## Create Validation Split

The Kaggle dataset only has Training/Testing. Create validation set from Training data.

In [ ]:
def create_validation_split(processed_dir, val_size=0.2, random_seed=42):
    """
    Create a validation split from the training data.
    
    Args:
        processed_dir: Path to processed dataset
        val_size: Proportion of training data to use for validation
        random_seed: Random seed for reproducibility
    """

    processed_path = Path(processed_dir)
    training_path = processed_path / 'Training'
    validation_path = processed_path / 'Validation'

    if not validation_path.exists():
        os.makedirs(validation_path, exist_ok=True)
        
    for cls in classes:
        class_path_train = training_path / cls
        class_path_val = validation_path / cls

        if not class_path_val.exists():
            os.makedirs(class_path_val, exist_ok=True)

        image_files = os.listdir(class_path_train)
        train_files, val_files = train_test_split(image_files, test_size=val_size, random_state=random_seed)

        for filename in val_files:
            src_path = class_path_train / filename
            dst_path = class_path_val / filename

            # move file to validation folder
            os.rename(src_path, dst_path)
    print("Validation split created.")
    

In [ ]:
create_validation_split(processed_dataset, val_size=0.2, random_seed=config['random_seed'])

In [ ]:
## Class Distribution After Preprocessing and Split
def compute_class_distribution(processed_dir):
    """
    Compute and display class distribution in each dataset split.
    
    Args:
        processed_dir: Path to processed dataset
    """

    processed_path = Path(processed_dir)

    for split in ['Training', 'Validation', 'Testing']:
        split_path = processed_path / split
        print(f"Class distribution in {split} set:")
        for cls in classes:
            class_path = split_path / cls
            count = len(os.listdir(class_path))
            print(f"  {cls}: {count} images")
        print()

compute_class_distribution(processed_dataset)

In [ ]:
def visualize_all_splits_distribution(processed_dir):
    """
    Visualize class distribution for all splits in a 2x2 grid.
    
    Args:
        processed_dir: Path to processed dataset
    """
    
    processed_path = Path(processed_dir)
    splits_to_plot = ['Training', 'Validation', 'Testing']
    
    # Create 2x2 grid (4th subplot will be empty or used for summary)
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    fig.suptitle('Class Distribution Across All Splits', fontsize=16, fontweight='bold')
    
    for idx, split in enumerate(splits_to_plot):
        split_path = processed_path / split
        
        if not split_path.exists():
            continue
            
        class_counts = {}
        for cls in classes:
            class_path = split_path / cls
            if class_path.exists():
                count = len([f for f in os.listdir(class_path) 
                           if not f.startswith('.')])
                class_counts[cls] = count
            else:
                class_counts[cls] = 0
        
        # Calculate position
        row = idx // 2
        col = idx % 2
        
        # Plot bar chart
        axes[row, col].bar(class_counts.keys(), class_counts.values(), color='steelblue')
        axes[row, col].set_xlabel('Class', fontsize=10)
        axes[row, col].set_ylabel('Number of Images', fontsize=10)
        axes[row, col].set_title(f'{split} Set', fontsize=12, fontweight='bold')
        axes[row, col].tick_params(axis='x', rotation=45)
        axes[row, col].grid(axis='y', alpha=0.3)
    
    # Use 4th subplot for total counts across all splits
    all_counts = {}
    for cls in classes:
        total = 0
        for split in splits_to_plot:
            class_path = processed_path / split / cls
            if class_path.exists():
                total += len([f for f in os.listdir(class_path) 
                            if not f.startswith('.')])
        all_counts[cls] = total
    
    axes[1, 1].bar(all_counts.keys(), all_counts.values(), color='coral')
    axes[1, 1].set_xlabel('Class', fontsize=10)
    axes[1, 1].set_ylabel('Number of Images', fontsize=10)
    axes[1, 1].set_title('Total (All Splits)', fontsize=12, fontweight='bold')
    axes[1, 1].tick_params(axis='x', rotation=45)
    axes[1, 1].grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Call the new function
visualize_all_splits_distribution(processed_dataset)

In [ ]:
def calculate_class_weights(processed_dir):
    """
    Calculate class weights for handling imbalance during training.
    
    Returns:
        tuple: (class_weights dict, class_counts dict)
    
    Class_weights are calculated as: total_samples / (num_classes * count_per_class)
    Class_weights dict maps class index to its weight.
    Class_counts dict maps class index to its sample count.
    """
    
    processed_path = Path(processed_dir)
    training_path = processed_path / 'Training'
    
    # Count samples per class in training set
    class_counts = {}
    for idx, cls in enumerate(classes):
        class_path = training_path / cls
        if class_path.exists():
            count = len([f for f in os.listdir(class_path) 
                        if not f.startswith('.')])
            class_counts[idx] = count
    
    # Calculate weights: total / (num_classes * count_per_class)
    total_samples = sum(class_counts.values())
    num_classes = len(class_counts)
    
    class_weights = {}
    for idx, count in class_counts.items():
        class_weights[idx] = total_samples / (num_classes * count)
    
    return class_weights, class_counts

def save_preprocessing_metadata(processed_dir, config, stats):
    """
    Save comprehensive preprocessing metadata for reproducibility.
    
    Args:
        processed_dir: Path to processed dataset
        config: Preprocessing configuration dictionary
        stats: Processing statistics from preprocessing
        
    Returns:
        Path to saved metadata file
    """
    
    processed_path = Path(processed_dir)
    
    # Calculate class weights
    class_weights, training_counts = calculate_class_weights(processed_dir)
    
    # Get distribution across all splits
    distribution = {}
    for split in ['Training', 'Validation', 'Testing']:
        split_path = processed_path / split
        if not split_path.exists():
            continue
            
        distribution[split] = {}
        for idx, cls in enumerate(classes):
            class_path = split_path / cls
            if class_path.exists():
                count = len([f for f in os.listdir(class_path) 
                           if not f.startswith('.')])
                distribution[split][cls] = count
    
    # Calculate imbalance ratio
    if training_counts:
        max_count = max(training_counts.values())
        min_count = min(training_counts.values())
        imbalance_ratio = max_count / min_count
        minority_idx = min(training_counts, key=training_counts.get)
        majority_idx = max(training_counts, key=training_counts.get)
    else:
        imbalance_ratio = 1.0
        minority_idx = 0
        majority_idx = 0
    
    # Create metadata dictionary
    metadata = {
        'preprocessing_info': {
            'timestamp': pd.Timestamp.now().isoformat(),
            'config': config,
            'total_processed': stats['total_images'],
            'duplicates_removed': stats['duplicate_count'],
            'failed_images': len(stats.get('failed', []))
        },
        'class_mapping': {
            str(idx): cls for idx, cls in enumerate(classes)
        },
        'class_distribution': distribution,
        'class_weights': {
            str(k): float(v) for k, v in class_weights.items()  # JSON serializable
        },
        'imbalance_info': {
            'imbalance_ratio': round(imbalance_ratio, 2),
            'minority_class': classes[minority_idx],
            'majority_class': classes[majority_idx],
            'recommendation': 'Use class-weighted loss function during training'
        },
        'dataset_structure': {
            'splits': ['Training', 'Validation', 'Testing'],
            'classes': classes,
            'image_size': config['target_size'],
            'format': 'grayscale',
            'preprocessing_applied': ['grayscale_conversion', 'resize_224x224', 'CLAHE']
        }
    }
    
    # Save to JSON
    metadata_path = processed_path / 'preprocessing_metadata.json'
    with open(metadata_path, 'w') as f:
        json.dump(metadata, f, indent=4)
    
    # Print summary
    print("\n" + "="*70)
    print("METADATA SAVED")
    print("="*70)
    print(f"\n📄 File: {metadata_path}")
    print(f"\n🎯 Class Weights (for model.fit()):")
    for idx, cls in enumerate(classes):
        weight = class_weights[idx]
        print(f"   Class {idx} ({cls:20s}): {weight:.4f}")
    
    print(f"\n⚖️  Imbalance Analysis:")
    print(f"   • Imbalance Ratio: {imbalance_ratio:.2f}:1")
    print(f"   • Minority Class: {classes[minority_idx]}")
    print(f"   • Majority Class: {classes[majority_idx]}")
    print(f"   • Recommendation: {metadata['imbalance_info']['recommendation']}")
    print("="*70 + "\n")
    
    return metadata_path

In [ ]:
def print_preprocessing_summary(stats, processed_dir):
    """Print comprehensive preprocessing summary"""
    
    print("\n" + "="*70)
    print("PREPROCESSING SUMMARY")
    print("="*70)
    
    print(f"\n📁 Output Directory: {processed_dir}")
    print(f"\n📊 Processing Statistics:")
    print(f"   • Total images processed: {stats['total_images']}")
    print(f"   • Duplicates skipped: {stats['duplicate_count']}")
    print(f"   • Failed: {len(stats.get('failed', []))}")
    
    if stats.get('failed'):
        print(f"\n⚠️  Failed Files:")
        for filename, error in stats['failed'][:5]:
            print(f"      - {filename}: {error}")
        if len(stats['failed']) > 5:
            print(f"      ... and {len(stats['failed']) - 5} more")
    
    print(f"\n📦 Images per Class:")
    for cls, count in stats['class_counts'].items():
        print(f"   • {cls:20s}: {count:4d} images")
    
    print(f"\n✅ Preprocessing Complete!")
    print("="*70 + "\n")

# Print summary
print_preprocessing_summary(stats, processed_dataset)

# Save metadata for training notebook
metadata_path = save_preprocessing_metadata(processed_dataset, config, stats)